In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [ ]:
train_df = pd.read_csv("hacktrain.csv")
test_df = pd.read_csv("hacktest.csv")


In [ ]:
print(train_df.info())
#print(train_df.head())
#print(test_df.info())
#print(test_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8000 non-null   int64  
 1   ID          8000 non-null   int64  
 2   class       8000 non-null   object 
 3   20150720_N  7440 non-null   float64
 4   20150602_N  6800 non-null   float64
 5   20150517_N  7200 non-null   float64
 6   20150501_N  7040 non-null   float64
 7   20150415_N  7520 non-null   float64
 8   20150330_N  6880 non-null   float64
 9   20150314_N  7280 non-null   float64
 10  20150226_N  6640 non-null   float64
 11  20150210_N  7360 non-null   float64
 12  20150125_N  6960 non-null   float64
 13  20150109_N  7120 non-null   float64
 14  20141117_N  6720 non-null   float64
 15  20141101_N  7600 non-null   float64
 16  20141016_N  6560 non-null   float64
 17  20140930_N  7200 non-null   float64
 18  20140813_N  7440 non-null   float64
 19  20140626_N  6400 non-null  

In [ ]:
class_counts = train_df['class'].value_counts()
print(class_counts)

class
forest        6159
farm           841
impervious     669
grass          196
water          105
orchard         30
Name: count, dtype: int64


In [ ]:
train_df.style

In [ ]:
# Dropping 'Unnamed: 0'
train_df.drop(columns=['Unnamed: 0'], inplace=True)
test_df.drop(columns=['Unnamed: 0'], inplace=True)

# Extracting features and target
X = train_df.drop(columns=['ID', 'class'])
y = train_df['class']
test_ids = test_df['ID']
X_test = test_df.drop(columns=['ID'])

# Imputing missing values with median..
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_test_imputed = imputer.transform(X_test)

# Encode target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [ ]:
class_counts = train_df['class'].value_counts()
print(class_counts)

class
forest        6159
farm           841
impervious     669
grass          196
water          105
orchard         30
Name: count, dtype: int64


In [ ]:
# Splitting the training data to evaluate the model
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_imputed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Training..
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_train_split, y_train_split)

# Predicting..
y_val_pred = model.predict(X_val_split)

# Decoding the labels back to original classes
y_val_true_labels = label_encoder.inverse_transform(y_val_split)
y_val_pred_labels = label_encoder.inverse_transform(y_val_pred)

# The classification report
from sklearn.metrics import classification_report
print(classification_report(y_val_true_labels, y_val_pred_labels))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

        farm       0.86      0.85      0.85       168
      forest       0.99      0.99      0.99      1232
       grass       0.71      0.56      0.63        39
  impervious       0.80      0.89      0.84       134
     orchard       0.50      0.33      0.40         6
       water       1.00      0.71      0.83        21

    accuracy                           0.95      1600
   macro avg       0.81      0.72      0.76      1600
weighted avg       0.95      0.95      0.95      1600



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
submission_df = pd.DataFrame({
    'ID': test_ids,
    'class': y_test_labels
})

submission_df.to_csv("submission.csv", index=False)
submission_df.head()

,ID,class
0,1,orchard
1,2,forest
2,3,orchard
3,4,forest
4,5,forest
